**Fashion MNIST using CNN**

As seen in the previous model using a basic neural network, the accuracy was coming around 88% which is low. The accuracy can be increased using a CNN (Convolutional Neural Network)

Importing the necessary libraries

In [1]:
#Various libraries for data handling, updation and visualisation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torchvision
import torch.optim as optim
from torchvision import datasets,transforms
from torch import nn
from torch.utils.data import DataLoader
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
import warnings
warnings.filterwarnings('ignore')
torch.manual_seed(42)

Loading the data

In [2]:
#Using torch to load the data
#Adding transforms
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
train_images = torchvision.datasets.FashionMNIST(
    root = './data/FashionMNIST',
    train = True,
    download = True,
    transform =transform
)
test_images = torchvision.datasets.FashionMNIST(
    root = './data/FashionMNIST',
    train = False,
    download = True,
    transform = transform
)
train_labels = train_images.targets
test_labels=test_images.targets
train_loader = torch.utils.data.DataLoader(train_images, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_images, batch_size=64, shuffle=False)

In [3]:
#Storing the labels
labels = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']



Exploring the data

In [4]:
#Function to explore data
def FMNIST_data_info(train_images, test_images):
    print("Train images length:", len(train_images))
    image, label = train_images[0]
    print("Image shape:", image.shape)
    print("Test images length:", len(test_images))
    image, label = test_images[0]
    print("Image shape:", image.shape)

In [5]:
#Calling the function to see data info
FMNIST_data_info(train_images, test_images)

Train images length: 60000
Image shape: torch.Size([1, 28, 28])
Test images length: 10000
Image shape: torch.Size([1, 28, 28])


From above, the training set is of 60000 images having 28X28 dimension and test set is of 10000 images having 28X28 dimension

Preprocessing of data

In [6]:
#Function for checking for null values of labels in test and train dataset as they can hamper with the results and cause redundancy
def count_null_values(labels):
    null_values = np.isnan(labels).sum()
    print("Null values:", null_values)

In [7]:
#Null values in Train_Labels
print("Null values in train_labels:")
count_null_values(train_labels)

#Null values in Test_Labels
print("Null values in test_labels:")
count_null_values(test_labels)

Null values in train_labels:
Null values: tensor(0)
Null values in test_labels:
Null values: tensor(0)


In [8]:
#Since there are no null values, we can carry on with the model

Building the model


In [9]:
#creating the model

class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()

        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.batchnorm1 = nn.BatchNorm2d(32)
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)
        self.dropout1 = nn.Dropout(0.3)

        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.batchnorm2 = nn.BatchNorm2d(64)
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)
        self.dropout2 = nn.Dropout(0.3)


        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.relu3 = nn.ReLU()
        self.batchnorm3 = nn.BatchNorm2d(128)
        self.maxpool3 = nn.MaxPool2d(kernel_size=2)
        self.dropout3 = nn.Dropout(0.25)

        self.flatten = nn.Flatten()

        self.fc1 = nn.Linear(128 * 3 * 3, 256)
        self.relu4 = nn.ReLU()
        self.dropout1 = nn.Dropout(0.5)

        self.fc2 = nn.Linear(256, 64)
        self.relu5 = nn.ReLU()
        self.dropout2 = nn.Dropout(0.5)

        self.fc3 = nn.Linear(64, 10)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.maxpool1(x)

        x = self.conv2(x)
        x = self.relu2(x)
        x = self.maxpool2(x)

        x = self.conv3(x)
        x = self.relu3(x)
        x = self.maxpool3(x)

        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu4(x)
        x = self.dropout1(x)

        x = self.fc2(x)
        x = self.relu5(x)
        x = self.dropout2(x)

        x = self.fc3(x)
        x = self.softmax(x)

        return x


In [10]:

#Creating an instance
model=CNNModel()
device = torch.device("cpu")
model.to(device)

CNNModel(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu1): ReLU()
  (batchnorm1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (maxpool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout1): Dropout(p=0.5, inplace=False)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu2): ReLU()
  (batchnorm2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (maxpool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout2): Dropout(p=0.5, inplace=False)
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu3): ReLU()
  (batchnorm3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (maxpool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout3): Dropout(p=0.25, inplace=False)
  (flatten): Fla

In [11]:
# Define the loss function
criterion = nn.CrossEntropyLoss()

# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [12]:
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    train_correct = 0
    total_samples = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass
        loss.backward()

        # Update the parameters
        optimizer.step()


        # Compute training statistics
        _, predicted = torch.max(outputs.data, 1)
        train_correct += (predicted == labels).sum().item()
        total_samples += labels.size(0)
        train_loss += loss.item()

    # Compute epoch statistics
    train_accuracy = 100 * train_correct / total_samples
    train_loss /= len(train_loader)

    # Print epoch results
    print(f"Epoch [{epoch+1}/{num_epochs}], "
          f"Train Loss: {train_loss:.4f}, "
          f"Train Accuracy: {train_accuracy:.2f}%")

Epoch [1/10], Train Loss: 1.7721, Train Accuracy: 69.01%
Epoch [2/10], Train Loss: 1.6708, Train Accuracy: 79.09%
Epoch [3/10], Train Loss: 1.6490, Train Accuracy: 81.27%
Epoch [4/10], Train Loss: 1.6243, Train Accuracy: 83.70%
Epoch [5/10], Train Loss: 1.6087, Train Accuracy: 85.27%
Epoch [6/10], Train Loss: 1.6038, Train Accuracy: 85.72%
Epoch [7/10], Train Loss: 1.6009, Train Accuracy: 86.03%
Epoch [8/10], Train Loss: 1.5975, Train Accuracy: 86.33%
Epoch [9/10], Train Loss: 1.5937, Train Accuracy: 86.73%
Epoch [10/10], Train Loss: 1.5920, Train Accuracy: 86.89%


In [13]:
import intel_extension_for_pytorch as ipex
model, optimizer = ipex.optimize(model, optimizer=optimizer)